In [ ]:
import sys
sys.path.insert(0,'/Users/Shared/c/CodeRepository/Formatting-Error-Correction/')
from datetime import datetime
from nltk.util import ngrams
from nltk.lm.preprocessing import padded_everygram_pipeline, pad_both_ends
from nltk.lm import KneserNeyInterpolated
from Scripts.S1_Corpus_Bigrams_Occurences import count_occur
import seaborn as sns
sns.set_theme()
sns.set_style("whitegrid",{'axes.grid' : False})
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd


In [ ]:
_,trainTok = count_occur('Ngrams_TrainingDataset/Training')
_,testTok = count_occur('Ngrams_TrainingDataset/Validation')

In [ ]:
order = [x for x in range(3,11)]
meanEntropy = {}
entropyScoresPerN = {}

In [ ]:
for ord in order:
    print(f'Starting time of training N-gram model with order {ord}: {datetime.now()}')

    trainingSet, vocab = padded_everygram_pipeline(ord, trainTok)
    kn = KneserNeyInterpolated(ord)
    kn.fit(trainingSet, vocab)

    # Testing each trained N-gram model
    currentScore = []
    for testFile in testTok:
        currTestPadded = list(pad_both_ends(testFile,n = ord))
        currTestFile = list(ngrams(currTestPadded, ord))
        crossEntropy = kn.entropy(currTestFile)
        currentScore.append(crossEntropy)
        
    # A dictionary that represents all entropy scores of each file that belongs to test set, per order n.
    entropyScoresPerN.update({ord:currentScore})
    # A dictionary that represents mean entropy score for all files that belong to test set, per order n.
    meanEntropy.update({ord:sum(currentScore)/len(currentScore)})
    print(f'N-gram order:{ord}, Avg Score on Test Set:{meanEntropy[ord]}, Time:{datetime.now()}\n')

In [ ]:
dataDF = pd.read_csv('/Users/Shared/c/CodeRepository/Formatting-Error-Correction/System_Evaluation/TestResults_EntropyScore.csv')

In [ ]:
fig = plt.figure(figsize = [8,6])
sns.boxplot(data = dataDF)
plt.yticks([0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5,5])
plt.xlabel('Order of N-gram')
plt.ylabel('Cross-Entropy');

In [ ]:
meanEntropy = {}
for i in list(dataDF.columns):
    meanEntropy.update({int(i):dataDF[i].mean()})

X = np.array(list(meanEntropy.keys()))
Y = np.array(list(meanEntropy.values()))

In [ ]:
plt.figure(figsize = [8,6])
plt.plot(X,Y)
plt.xlabel('Order of N-gram')
plt.ylabel('Mean Cross-Entropy')
plt.yticks([0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6]);